In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from tensorflow.keras.optimizers import Adam

In [4]:
from tensorflow.keras.applications import VGG19, VGG16

In [6]:
import cv2
import os 
import pickle

In [8]:
from imutils import paths

In [9]:
LABELS = set(["Crown and Root Rot", "Healthy Wheat", "Leaf Rust", "Wheat Loose Smut"])

In [10]:
count = 0 
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [11]:
image = cv2.imread('Images/Crown and Root Rot/00041.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (224, 224))

In [12]:
count = 0
for images in paths.list_images('Images/Crown and Root Rot/'):
    print(images)
    count = count + 1
    if count == 5:
        break

Images/Crown and Root Rot/00041.jpg
Images/Crown and Root Rot/00051.jpg
Images/Crown and Root Rot/00061.jpg
Images/Crown and Root Rot/00071.jpg
Images/Crown and Root Rot/00081.png


In [13]:
sample_path = list(paths.list_images('Images/Crown and Root Rot/'))[0]

In [14]:
sample_path

'Images/Crown and Root Rot/00041.jpg'

In [15]:
sample_path.split('/')

['Images', 'Crown and Root Rot', '00041.jpg']

In [16]:
os.listdir()

['.git',
 '.gitattributes',
 '.ipynb_checkpoints',
 'Images',
 'README.md',
 'Wheat Disease Detection.ipynb']

In [17]:
Labels = os.listdir('Images')

In [18]:
print(Labels)

['Crown and Root Rot', 'Healthy Wheat', 'Leaf Rust', 'Wheat Loose Smut']


In [19]:
CROWN_AND_ROOT_ROT_PATH = 'Images/Crown and Root Rot/'
HEALTHY_AND_WHEAT_PATH = 'Images/Healthy Wheat/'
LEAF_RUST_PATH = 'Images/Leaf Rust/'
WHEAT_LOOSE_SMUT_PATH = 'Images/Wheat Loose Smut/'

In [23]:
from tqdm import tqdm

In [24]:
data = []
labels = []
for image_path in tqdm(paths.list_images(CROWN_AND_ROOT_ROT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(HEALTHY_AND_WHEAT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)

for image_path in tqdm(paths.list_images(LEAF_RUST_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
for image_path in tqdm(paths.list_images(WHEAT_LOOSE_SMUT_PATH)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    
    label = image_path.split('/')[-2]
    
    labels.append(label)
    
    

1021it [00:04, 205.64it/s]
1146it [00:26, 43.57it/s] 
1286it [01:39, 12.90it/s] 
930it [00:05, 178.97it/s]


In [25]:
labels[-1]

'Wheat Loose Smut'

In [38]:
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [39]:
headmodel = VGG19(weights = "imagenet", include_top = False,
             input_tensor = Input(shape = (224, 224, 3)))
model = headmodel.output
model = AveragePooling2D(pool_size= (5, 5))(model)
model = Flatten(name = 'flatten')(model)
model = Dense(512, activation = 'relu')(model)
model = Dropout(0.2)(model)
model = Dense(len(Labels), activation = 'softmax')(model)

final_model = Model(inputs = headmodel.input, outputs = model)

for layer in headmodel.layers:
    layer.trainable = False

In [40]:
opt = Adam(lr = 1e-3)
final_model.compile(loss = "categorical_crossentropy", optimizer = opt,
                   metrics = ["accuracy"])

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [42]:
data = np.array(data)
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [43]:
labels[0]

array([1, 0, 0, 0], dtype=int32)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2,
                                                    stratify = labels, random_state = 101)

In [45]:
len(X_train)

3506

In [53]:
import tensorflow as tf

In [52]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [49]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13337211135697591325
]


In [51]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [37]:
final_model.fit(X_train, y_train, validation_data = (X_test, y_test), 
               epochs = 5, batch_size = 64)

Epoch 1/5
 9/55 [===>..........................] - ETA: 3:58 - loss: 2.3979 - accuracy: 0.5260

KeyboardInterrupt: 